
# Time-Series Sensor Forecasting using TinyML (Steps 1–3)

This notebook demonstrates:
1. Time-series sensor data generation
2. Training a tiny neural network for forecasting
3. Converting the model to TensorFlow Lite (MCU-ready)

Target: Embedded systems (STM32 / ESP32)


## Step 1: Generate Synthetic Sensor Time-Series Data

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

time = np.arange(0, 100, 0.1)
temperature = 25 + 2*np.sin(time) + 0.1*np.random.randn(len(time))

plt.figure()
plt.plot(time, temperature)
plt.title("Synthetic Temperature Sensor Data")
plt.xlabel("Time")
plt.ylabel("Temperature (°C)")
plt.show()


## Step 2: Create Sliding Window Dataset

In [ ]:

WINDOW_SIZE = 10
X, y = [], []

for i in range(len(temperature) - WINDOW_SIZE):
    X.append(temperature[i:i+WINDOW_SIZE])
    y.append(temperature[i+WINDOW_SIZE])

X = np.array(X)
y = np.array(y)

print("Input shape:", X.shape)
print("Output shape:", y.shape)


## Step 3: Train a Tiny Neural Network Model

In [ ]:

import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=(WINDOW_SIZE,)),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

history = model.fit(X, y, epochs=20, batch_size=16)


## Training Loss Curve

In [ ]:

plt.figure()
plt.plot(history.history['loss'])
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.show()


## Actual vs Predicted Sensor Values

In [ ]:

predictions = model.predict(X[:100])

plt.figure()
plt.plot(y[:100], label="Actual")
plt.plot(predictions.flatten(), label="Predicted")
plt.legend()
plt.title("Actual vs Predicted Sensor Values")
plt.show()


## Convert Model to TensorFlow Lite

In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("sensor_forecast.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model size:", len(tflite_model), "bytes")
